## 1. El camino de epidemia a pandemia
<p><img style="float: left; margin:5px 20px 5px 1px; width:40%" src="https://www.nps.gov/aboutus/news/images/CDC-coronavirus-image-23311-for-web.jpg?maxwidth=650&autorotate=false"></p>
<p>En diciembre de 2019, la enfermedad del coronavirus fue identificada por primera vez en Wuhan, region de China. Para el 11 de marzo de 2020, la Organización Mundial de la Salud (OMS) ya había categorizado al COVID-19 como una pandemia. Muchas cosas han pasado desde entonces con brotes masivos en Iran, Corea del Sur e Italia. </p>
<p>Ahora sabemos que el COVID-19 se contagia por fluidos respiratorios en actividades tales como tocer, estornudar o hablar. Aún así, es importante analizar lo rápido que el virus se propagó alrededor del mundo. Con esta información, ¿podemos ver algun efecto de las políticas de salud implementadas, como cuarentenas y cierres temporales? </p>
<p>Afortunadamente, algunas organizaciones alrededor del mundo han recolectado datos para que los gobiernos puedan monitorear el comportamiento global de la pandemia. Es notable el trabajo del Centro de Ciencias en Sistemas e Ingeniería de la Universidad Johns Hopkins para crear un <a href="https://github.com/RamiKrispin/coronavirus">repositorio público con los datos disponibles</a> que reúne la información de la OMS, los Centros de Control de enfermedades y Secretarías o Ministerios de Salud de diversos países.</p>
<p>En este proyecto, crearás vizualizaciones de los datos desde las primeras semanas del COVID-19 hasta el contagio mundial, con el objetivo de identificar el punto en el que el virus se convirtió en pandemia.</p>
<p><em>Los datos que utilizaremos tienen como fecha de corte el 17 de Marzo de 2020. Puedes intentar replicar este análisis con los datos más actualizados.</em></p>

In [110]:
# Carga los paquetes ggplot2 y dplyr\nlibrary(readr)\nlibrary(ggplot2)\nlibrary(dplyr)\n\n# Lee la base datos/casos_mundo.csv en el objeto casos_mundiales\casos_mundiales <- read_csv(\"datos/casos_mundo.csv\")\n\n# Ver el resultado\casos_mundiales

## 2. Casos confirmados a nivel mundial
<p>La tabla anterior muestra los casos acumulados confirmados de COVID-19 en todo el mundo por fecha. Al sólo leer los números en una tabla hace que sea difícil tener una idea de la escala y el crecimiento del brote. Dibujemos una gráfica con geometría de línea para visualizar los casos confirmados en todo el mundo.\nMapea la variable 'casos_acum' en el eje y, así como 'fecha' en el eje x. 
Añade la etiqueta del eje y para que diga 'Casos acumulados confirmados'</p>

In [112]:
# Crea una gráfica de línea con 'casos_acum' y 'fecha'
# Etiqueta el eje y
ggplot(casos_mundiales, aes(x=fecha, y=casos_acum)) +
  geom_line() +
  labs(y="Casos acumulados confirmados")

## 3. Comparando China con el mundo
<p>Nuestro primer gráfico es alarmante con cerca de 200 mil casos confirmados a nivel mundial. Adicionalmente, algo extraño está pasando: hay un salto raro a mediados de Febrero, despues la tasa de casos desciende un poco para volver a acelerarse en Marzo. Necesitamos profundizar en nuestro análisis para entender lo que está sucediendo.</p>
<p>En la fase temprana del contagio, los casos de COVID-19 estaban centrados primordialmente en China. Creemos una gráfica que muestre los casos confirmados en China separándolo de la información del resto del mundo para ver si nos da alguna pista.</p>
<p><em>En las tareas siguientes utilizaremos este gráfico, por lo que es importante que los datos de mapeo (aesthetics) los añadamos en la línea de la geometría, en lugar de hacerlo de forma global dentro de la función ggplot().</em></p>

In [114]:
# Lee la base en datos/casos_china_vs_mundo.csv
casos_china_vs_mundo <- read_csv("datos/casos_china_vs_mundo.csv")

# Ver los resultados
casos_china_vs_mundo

# Grafica de nuevo los casos acumulados por fecha (como en la tarea anterior)
# Mapea la variable es_china en la dimensión 'color'.
# Define el mapeo dentro de la geometría de línea
graf_casos_china_vs_mundo <- ggplot(casos_china_vs_mundo) +
  geom_line(aes(x=fecha, y=casos_acum, color=es_china)) +
  ylab("Casos acumulados confirmados")

# Ver el gráfico
graf_casos_china_vs_mundo

## 4. ¡Hagamos algunas anotaciones!
<p>Ambas lineas tienen formas muy distintas entre sí. En febrero, la mayoría de los casos estaban en China. Eso cambió en Marzo cuando se convirtió en un problema global: alrededor del 14 de Marzo, el total de casos confirmados fuera de China sobrepasaron los casos internos. Este fue el día que la OMS declaró la enfermedad como pandemia.</p>
<p>Existen otros eventos relevantes que pasaron durante la fase de contagio. Por ejemplo, el gran salto en la serie de China el 13 de febrero no se debe a un alto número de contagios, sino a un cambio en la metodología de reporte (incluyeron las tomografías como evidencia del COVID-19, en lugar de sólo las pruebas de laboratorio).</p>
<p>Al hacer anotaciones a nuestra gráfica de eventos como este, podemos interpretar de mejor forma la serie.</p>

In [116]:
eventos_oms <- read.csv('datos/eventos_oms.csv', stringsAsFactors = FALSE)
) %>%
  mutate(fecha = as.Date(fecha))

# Usando la base eventos_oms, añade lineas punteadas verticales mapeando 'fecha' en la dimensión xintercept
# y añade también una geometría de texto (geom_text) mapeando en x 'fecha'
# en label 'evento' y estableciendo la dimensión y a una altura fija de 100,000
graf_casos_china_vs_mundo +
  geom_vline(aes(xintercept=fecha), data=eventos_oms, linetype="dashed") +
  geom_text(aes(x=fecha, label=evento), y=100000, data=eventos_oms)

## 5. Añadir línea de tendencia a China
<p>Para determinar la gravedad de la situación, calcularemos la velocidad de crecimiento de los casos confirmados. Un buen inicio sería determinar si los casos crecen de forma lineal o no.</p>
<p>Hay un claro aumento de casos el 13 de febrero a partir de los cambios en el reporte de China. Sin embargo, unos días después, la velocidad de crecimiento vuelve a caer. ¿Cómo podemos describir el comportamiento de los casos confirmados en China después de l 15 de febrero?</p>

In [118]:
# Filtra la base para quedarnos con las observaciones que en la variable 'es_china' tienen el valor 'China' desde el 15 de febrero
china_post_15feb <- casos_china_vs_mundo %>%
  dplyr::filter(fecha >= "2020-02-15" & es_china =="China")

# Usando china_post_15feb, genera una gráfica de línea con casos_acum vs. fecha
# Añade una línea de regresión lineal, sin áreas de error 
ggplot(china_post_15feb, aes(x=fecha, y=casos_acum, color=es_china)) +
  geom_line() +
  geom_smooth(method="lm", se=FALSE) +
  ylab("Casos confirmados acumulados")

## 6. ¿Y el resto del mundo?
<p>De la gráfica anterior observamos que la tasa de crecimiento en China es más lenta que el crecimiento lineal. Es una gran noticia porque indica que China al menos contuvo el virus a fines de febrero y principios de marzo.</p>
<p>¿Cómo le fue al resto del mundo cuando lo comparamos con el crecimiento lineal?</p>

In [120]:
# Filtra los casos_china_vs_mundo para quedarte con las observaciones donde 'es_china' sea igual a 'No es China'
no_china <- dplyr::filter(casos_china_vs_mundo, es_china == "No es China")

# Usando no_china, dibuja un gráfico de línea con casos_acum vs. fecha
# Añade una regresión lineal, sin barras de error
graf_no_china <- ggplot(no_china, aes(x=fecha, y=casos_acum)) +
  geom_line() +
  geom_smooth(method="lm", se=FALSE) +
  ylab("Casos confirmados acumulados")

# Imprime el resultado
graf_no_china 

## 7. Cambiar a escala logarítmica
<p>En el gráfico anterior, podemos ver que una línea recta no se ajusta bien porque el resto del mundo está creciendo mucho más rápido que un crecimiento lineal. Agregemos la escala logarítmica al eje. Para ello añade scale_y_log10() al gráfico.</p>

In [122]:
# Modiufica el gráfico para usar la escala logarítmica en el eje y
graf_no_china + 
  scale_y_log10()

## 8. ¿Cuales paises han sido los más afectados?
<p>Con la escala logarítmica, encontramos un mejor modelo para describir nuestros datos. Desafortunadamente, desde el punto de vista de salud pública, esto significa que los casos de COVID-19 crecieron a una tasa exponencial.</p>
<p>No todos los países han sido afectados de igual forma, por lo que sería útil tener una gráfica para poder diferenciarlos. Hagamos seis gráficas con los casos de los 7 países más afectados por el COVID-19.</p>

In [124]:
# Carga la base de datos por país
casos_por_pais <- read_csv("datos/cases_por_pais.csv")
glimpse(cases_por_pais)

# Agrupa por país, utiliza summarize para calcular casos_totales como el valor máximo de cada Estado y encuentra los primeros 7
primeros_lugares <- casos_por_pais %>%
  group_by(pais) %>%
  summarize(casos_totales = max(casos_acum)) %>% 
  top_n(6, casos_totales)

# Ve los resultados
primeros_lugares

## 9. Grafica los países más afectados en mediados de marzo
<p>Aunque el brote se identificó por primera vez en China, solo hay un país de Asia oriental (Corea del Sur) en la tabla anterior. Cuatro de los países enumerados (Francia, Alemania, Italia y España) están en Europa y comparten fronteras. Para obtener más contexto, podemos trazar los casos confirmados de estos países a lo largo del tiempo.</p><p>En el último ejercicio utiliza la funcion facet_wrap() para generar las seis gráficas de los países más afectados.</p>

In [126]:
# Carga las bases del top 7
casos_top7 <- read_csv("datos/casos_top7.csv")

# 
glimpse(casos_top7)

# Usando casos_top7, dibuja un gráfico de linea de
# casos_acum vs. fecha, coloreado por país, en páneles separados.
ggplot(casos_top7,
      aes(x=fecha, y=casos_acum, color=pais))+
geom_line()+
      facet_wrap(~pais) +
labs(y="Casos confirmadosacumulados")